## Final Project Submission

Please fill out:
* Student name: Joseph Husney
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: James Irving
* Blog post URL:


In [95]:
# import modules needed for project
import requests
import pandas as pd
from tqdm import tqdm

In [96]:
# Get keys in dict
for key in popular_movies.keys():
    print(key)
    value = response.json()[key] #Use standard dictionary formatting
    print(type(value)) #What type is it?
    print('\n\n') #Separate out data

page


KeyError: 'page'

In [ ]:
# see sample of the key 'results'
popular_movies['results'][:2]

In [ ]:
type(popular_movies['results'])

In [ ]:
#put all results into dataframe
df = pd.DataFrame.from_dict(popular_movies['results'])
df.head(50)

In [ ]:
# put all ids into a list
print(len(df))
print(df.columns)
movie_ids = []
for index, rows in df.iterrows():
    my_list = [rows.id]
    movie_ids.append(my_list)
print(movie_ids)

In [ ]:
movie_ids[0][0]

In [ ]:
# trying to get information on all movie id's that I got from popular movie sample
movie_info = []
for id_ in movie_ids:
    url = f"https://api.themoviedb.org/3/movie/{id_[0]}?api_key=34017996022f64249a86491295577f6b&language=en-US"
    response = requests.get(url)
    movie_info.append(response.json())
movie_info

In [ ]:
df = pd.DataFrame.from_dict(movie_info)
df.head()

In [ ]:
# Get all movie id's before 2019
df = pd.read_csv('zippedData/tmdb.movies.csv.gz')
print(len(df))
print(df.columns)
movie_ids = []
for index, rows in df.iterrows():
    if int(df['release_date'][0].split('-')[0]) < 2019:
        my_list = [rows.id]
        movie_ids.append(my_list)
print(movie_ids)

In [97]:
# Getting so far info about 1000 movies based on 1000 id's that I got from dataset i was given
# There are 26 thousand movies but it takes very long
movie_info = []
for id_ in tqdm(movie_ids[:1000]):
    url = f"https://api.themoviedb.org/3/movie/{id_[0]}?api_key=34017996022f64249a86491295577f6b&language=en-US"
    response = requests.get(url)
    movie_info.append(response.json())
df = pd.DataFrame.from_dict(movie_info)




  0%|                                                 | 0/1000 [00:00<?, ?it/s]


  0%|                                         | 1/1000 [00:00<02:56,  5.66it/s]


  0%|                                         | 2/1000 [00:00<02:41,  6.19it/s]


  0%|                                         | 3/1000 [00:00<02:32,  6.54it/s]


  0%|▏                                        | 4/1000 [00:00<02:22,  7.01it/s]


  0%|▏                                        | 5/1000 [00:00<02:21,  7.03it/s]


  1%|▏                                        | 6/1000 [00:00<02:19,  7.14it/s]


  1%|▎                                        | 7/1000 [00:00<02:17,  7.21it/s]


  1%|▎                                        | 8/1000 [00:01<02:20,  7.06it/s]


  1%|▎                                        | 9/1000 [00:01<02:17,  7.21it/s]


  1%|▍                                       | 10/1000 [00:01<02:13,  7.42it/s]


  1%|▍                                       | 11/1000 [00:01<02:08,  7.68it/s]


  1%|▍       

 10%|███▉                                    | 99/1000 [00:12<02:02,  7.38it/s]


 10%|███▉                                   | 100/1000 [00:13<01:57,  7.65it/s]


 10%|███▉                                   | 101/1000 [00:13<02:09,  6.95it/s]


 10%|███▉                                   | 102/1000 [00:13<02:06,  7.10it/s]


 10%|████                                   | 103/1000 [00:13<02:04,  7.18it/s]

KeyboardInterrupt: 

In [ ]:
# Telling pandas to allow me to see all rows and columns without summarizing
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [ ]:
df_nulls = df.isna()['release_date']
df_nulls = df_nulls[lambda x: x == True]
df_nulls
# should get
# 182    True
# 864    True
# need to delete those rows becase they aren't allowing me to view what I want to view